# 題目：針對老師評論進行情感分析

# 先整理資料，分別有db裡的跟爬來的

# 留下老師與評論跟課程

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import jieba

In [ ]:
data = pd.read_table('teacher_data')
engine = create_engine('sqlite:///database.db')

In [ ]:
names = []
for i in data.values:
    if ' ' not in i[0][0]:
        if '-' in i[0]:
            name = i[0].split('-')
            name = name[0]
            names.append(name)

In [ ]:
names

In [ ]:
count = 0
comment_df = pd.DataFrame(columns = ["teacher", "comment","course"])
for j,i in enumerate(data.values):
    comment = i[0]
    course = ''
    if names[count] in comment:
        count += 1
    comment = comment.split('(')[0]
    if '-' in comment:
        comment = comment.split('-')[1]
    if ']' in comment:
        course = comment.split(']')[0]
        course = course.split('[')[1]
        comment = comment.split(']')[1]
    comment = comment.strip()
    if course == '':
        comment_df.loc[j] = [names[count-1],comment,pd.NaT]
    else:
        comment_df.loc[j] = [names[count-1],comment,course]
    print (names[count-1] + '-------' + comment + '----' + course)

In [ ]:
comment_df

In [ ]:
data = pd.read_sql('select * from teacher',con=engine)
data = data.drop('star',axis=1)

In [ ]:
data

In [ ]:
data = data.drop(['department','c','b'],axis=1)

In [ ]:
data = data.rename(columns={'a':'comment'})

In [ ]:
data

In [ ]:
comment_df = pd.concat([data,comment_df],ignore_index=True)

In [ ]:
comment_df

In [ ]:
data = pd.read_sql('select * from teacher',con=engine)
data = data.drop('star',axis=1)

In [ ]:
data

In [ ]:
data = data.drop(['department','c','a'],axis=1)

In [ ]:
data = data.rename(columns={'b':'comment'})

In [ ]:
data

In [ ]:
comment_df = pd.concat([data,comment_df],ignore_index=True)

In [ ]:
data = pd.read_sql('select * from teacher',con=engine)
data = data.drop('star',axis=1)

In [ ]:
data

In [ ]:
data = data.drop(['department','a','b'],axis=1)

In [ ]:
data = data.rename(columns={'c':'comment'})

In [ ]:
data

In [ ]:
comment_df = pd.concat([data,comment_df],ignore_index=True)

In [ ]:
comment_df

In [ ]:
index = comment_df[comment_df['comment'] == 'null'].index

In [ ]:
for i in index:
    comment_df = comment_df.drop(i,axis=0)

In [ ]:
comment_df[comment_df['teacher'] == '林國貴']

In [ ]:
comment_df

In [ ]:
comment_df.to_csv('comment.csv',index=0)

# 開始做情感分析，判斷對老師的評價

# 把一些特殊詞彙加進字庫裡，如：佛、推之類的

In [2]:
data = pd.read_csv('comment.csv')

In [3]:
data

,comment,course,teacher
0,這門課滿像演講，不用考試，老師常常跟我們分享他去阿拉伯工作的見聞，偶爾也會邀請一些在國外工作...,阿拉伯世界-生活與經貿,詹授合
1,不難，可以不去上課，最後一節繳交作業兼點名即可。,通識-毒化物,忘了抱歉
2,人真的很好,實用營養學,何麗齡
3,不用什麼基礎，人來就好，老師人很好，都是第二節上課才傳點名單 ，偶爾會直接點名,國際關係,林賢治
4,給分都很高,行政學,黎淑慧
5,就聽老師講一些儒道思想，他也會問同學們哪裡有問題還有感想，所以要認真聽喔,儒道思想學情緒管理,王祖淼
6,壓力不大,智慧城市與行動應用,陳奕靜
7,有點類似高中的公民,經濟與生活,陳俊卿
8,只要認真聽就會了，考前老師的複習一定要聽，回去看一看就90幾了,營養與健康,何麗齡
9,廢課一節，唯一有趣的是每節老師上課穿的衣服都很特別。,世界音樂文化,盧韋聿


In [4]:
sentiment_df = pd.read_csv('SentimentDict.csv')

In [5]:
sentiment_df

,positive,negative,not,degree-1,degree-2,degree-3,degree-4,degree-5,degree-6
0,愛,搥胸頓足,不,百分之百,不過,大不了,點點滴滴,半點,不為過
1,愛不忍釋,哀,不一定,倍加,不少,多,多多少少,不大,超
2,愛不釋手,哀哀切切,不了,備至,不勝,更,怪,不丁點兒,超額
3,愛寵,哀愁,不了了之,不得了,慘,更加,好生,不甚,超外差
4,愛戴,哀憐,不力,不堪,沉,更進一步,還,不怎麼,超微結構
5,愛撫,哀憫,不久,不可開交,沉沉,更為,或多或少,聊,超物質
6,愛好,哀慼,不切實際,不亦樂乎,出奇,還,略,沒怎麼,出頭
7,愛護,哀悽,不及,不折不扣,大為,還要,略加,輕度,多
8,愛憐,哀切,不少,徹頭徹尾,多,較,略略,弱,浮
9,愛戀,哀傷,不支,充分,多多,較比,略微,絲毫,過


In [6]:
positives_set = set(sentiment_df['positive'])

In [7]:
positives_set

{'牢穩',
 '白紙黑字',
 '風光旖旎',
 '風風光光',
 '喜盈盈',
 '不動搖',
 '繁花似錦',
 '姝',
 '無愧',
 '令人大開眼界',
 '光潤',
 '灼亮',
 '盡忠',
 '熟透',
 '柔中有剛',
 '服服帖帖',
 '名譽',
 '有禮貌',
 '認可',
 '得體',
 '樂不可支',
 '亮閃閃',
 '清幽幽',
 '平平安安',
 '全人',
 '屹立',
 '脣紅齒白',
 '穠豔',
 '高看',
 '理想',
 '小',
 '碩學',
 '玉潔冰清',
 '稱心如願',
 '敏捷',
 '坦誠相見',
 '歷歷',
 '示同意',
 '脈脈含情',
 '把牢',
 '寂靜',
 '泰然處之',
 '攻得',
 '端雅',
 '文武全才',
 '神氣',
 '狂熱',
 '文靜',
 '快刀斬亂麻',
 '善目',
 '不容懷疑',
 '最理想',
 '婉娩',
 '結結實實',
 '從頭至尾',
 '千依百順',
 '明澈',
 '豐',
 '稱謝',
 '蕃',
 '宜人',
 '懂禮',
 '溫潤',
 '敬奉',
 '不無道理',
 '明面上',
 '通達',
 '祺',
 '聰明伶俐',
 '有意',
 '精衛填海',
 '大師',
 '味道鮮',
 '有理',
 '無可厚非',
 '英俊',
 '致敬',
 '外向',
 '如意',
 '良',
 '歡忭',
 '便利',
 '狀況良好',
 '景氣',
 '明處',
 '天姿國色',
 '費心',
 '多姿多采',
 '明媚',
 '講',
 '寫意',
 '清香',
 '心慈',
 '精力充沛',
 '殷切',
 '澄明',
 '翕然',
 '經',
 '敬愛',
 '踏踏實實',
 '鐵面無私',
 '謝',
 '相助',
 '不屈不撓',
 '俊美',
 '樂滋滋',
 '喜眉笑眼',
 '經久耐用',
 '起義',
 '滿懷豪情',
 '不苟',
 '欣然',
 '生機盎然',
 '動人心魄',
 '客觀上',
 '熱',
 '默許',
 '放心',
 '承情',
 '鬆動',
 '蔚',
 '氣度恢宏',
 '岑寂',
 '貼切',
 '善用',
 '明快生動',
 '堅忍',
 '閃亮',
 '扎眼',
 '晟',
 '骨鯁',
 '著

In [8]:
negatives_set = set(sentiment_df['negative'])

In [9]:
negatives_set

{nan,
 '惰性',
 '憤世嫉俗',
 '心眼兒窄',
 '剛愎',
 '黷',
 '優柔寡斷',
 '黑心腸',
 '忿恚',
 '惻然',
 '低三下四',
 '唾棄',
 '出氣',
 '造作',
 '荒唐無稽',
 '吝',
 '與世無爭',
 '詛',
 '嗔怪',
 '失敗',
 '粗手笨腳',
 '詰屈聱牙',
 '心猿意馬',
 '傖',
 '鳴冤叫屈',
 '含糊不清',
 '哀悽',
 '作色',
 '越禮',
 '帶有敵意',
 '厭膩',
 '差可',
 '發怒',
 '單',
 '原宥',
 '懶人',
 '看扁',
 '悲慘',
 '有失身分',
 '不堪入耳',
 '亂紛紛',
 '瑣屑',
 '與世隔絕',
 '含蓄',
 '心黑手辣',
 '詘',
 '下',
 '淫穢',
 '私下',
 '怍',
 '沒有頭腦',
 '窶',
 '零亂',
 '保守',
 '恚恨',
 '無關痛癢',
 '泛',
 '瘦',
 '發膩',
 '孛',
 '亂象',
 '貧窮',
 '孑然',
 '過不去',
 '喧噪',
 '厭煩',
 '眼皮子高',
 '震',
 '灰頭灰臉',
 '生氣',
 '氣',
 '枯寂',
 '殺氣騰騰',
 '冷汗',
 '自咎',
 '男盜女娼',
 '悔不當初',
 '憤懣',
 '沉滯',
 '厭世',
 '粗野',
 '不盡',
 '魔鬼',
 '暴跳如雷',
 '不羈',
 '隱身',
 '說',
 '鐵血',
 '黯然',
 '踩壞',
 '惶悚',
 '勢不兩立',
 '詐領',
 '恨入骨髓',
 '凜然',
 '選錯',
 '迷濛蒙',
 '小',
 '機械',
 '粗俗',
 '繁複',
 '危如累卵',
 '不得勁',
 '誕',
 '陰森森',
 '厝火積薪',
 '冷感',
 '淡而無味',
 '滑頭滑腦',
 '莽蒼',
 '挑逗性',
 '貪嗔',
 '惼',
 '傻傻',
 '嗤之以鼻',
 '稀鬆平常',
 '鄙薄',
 '急死',
 '黑洞洞',
 '平鋪直敘',
 '險峻',
 '狎',
 '繁亂',
 '虛榮',
 '擅闖',
 '壞情',
 '系',
 '令人遺憾',
 '瑣細',
 '讆',
 '鄙',
 '變心',
 '背靜',
 '唾罵',
 '揹人',
 '

In [10]:
not_set = set(sentiment_df['not'])

In [11]:
not_set

{nan,
 '不',
 '不一定',
 '不久',
 '不了',
 '不了了之',
 '不亦',
 '不但',
 '不住',
 '不佳',
 '不便',
 '不停',
 '不僅',
 '不克',
 '不免',
 '不全',
 '不再',
 '不准',
 '不切實際',
 '不利',
 '不力',
 '不及',
 '不只',
 '不可',
 '不可不',
 '不可以',
 '不可思議',
 '不可能',
 '不合',
 '不吉',
 '不同',
 '不和',
 '不外',
 '不夠',
 '不奈',
 '不好',
 '不如',
 '不妨',
 '不安',
 '不定',
 '不宜',
 '不客氣',
 '不容置疑',
 '不對',
 '不對勁',
 '不少',
 '不屑',
 '不巧',
 '不常',
 '不平',
 '不幸',
 '不幹',
 '不得不',
 '不必',
 '不快',
 '不怕',
 '不悅',
 '不惜',
 '不愧',
 '不懂',
 '不應該',
 '不拘小節',
 '不支',
 '不料',
 '不斷',
 '不早',
 '不明',
 '不是',
 '不時',
 '不會',
 '不服',
 '不朽',
 '不止',
 '不比',
 '不決',
 '不清',
 '不清不楚',
 '不測',
 '不滿',
 '不濟',
 '不為',
 '不無',
 '不然',
 '不爽',
 '不獨',
 '不理',
 '不甘',
 '不甚',
 '不用',
 '不由',
 '不當',
 '不疑',
 '不瞞',
 '不知',
 '不知所措',
 '不祥',
 '不禁',
 '不符',
 '不算',
 '不管',
 '不耐',
 '不耐煩',
 '不肖',
 '不肯',
 '不能',
 '不致於',
 '不良',
 '不虞',
 '不行',
 '不要',
 '不覺',
 '不解',
 '不計',
 '不計其數',
 '不許',
 '不該',
 '不詳',
 '不論',
 '不謂',
 '不講',
 '不讓',
 '不賴',
 '不足',
 '不通',
 '不過',
 '不適',
 '不配',
 '不錯',
 '不難',
 '不需',
 '不順',
 '不須',
 '不願',
 '不顧',
 '並非',
 '切勿',
 '別',
 '反',
 '否',
 '否定',

In [12]:
degree_dict = {}
for word in sentiment_df['degree-1']:
    degree_dict[word] = 1.8

In [13]:
for word in sentiment_df['degree-2']:
    degree_dict[word] = 1.6

In [14]:
for word in sentiment_df['degree-3']:
    degree_dict[word] = 1.4

In [15]:
for word in sentiment_df['degree-4']:
    degree_dict[word] = 1.2

In [16]:
for word in sentiment_df['degree-5']:
    degree_dict[word] = 1.1

In [17]:
for word in sentiment_df['degree-6']:
    degree_dict[word] = 1.5

In [18]:
degree_dict

{'百分之百': 1.8,
 '倍加': 1.8,
 '備至': 1.8,
 '不得了': 1.8,
 '不堪': 1.8,
 '不可開交': 1.8,
 '不亦樂乎': 1.8,
 '不折不扣': 1.8,
 '徹頭徹尾': 1.8,
 '充分': 1.8,
 '到頭': 1.8,
 '地地道道': 1.8,
 '非常': 1.8,
 '極': 1.8,
 '極度': 1.8,
 '極端': 1.8,
 '極其': 1.8,
 '極為': 1.8,
 '截然': 1.8,
 '盡': 1.8,
 '驚人地': 1.8,
 '絕': 1.8,
 '絕頂': 1.8,
 '絕對': 1.8,
 '絕對化': 1.8,
 '刻骨': 1.8,
 '酷': 1.8,
 '滿': 1.8,
 '滿貫': 1.8,
 '滿心': 1.8,
 '莫大': 1.8,
 '奇': 1.8,
 '入骨': 1.8,
 '甚為': 1.8,
 '十二分': 1.8,
 '十分': 1.8,
 '十足': 1.8,
 '死': 1.8,
 '滔天': 1.8,
 '痛': 1.8,
 '透': 1.8,
 '完全': 1.8,
 '完完全全': 1.8,
 '萬': 1.8,
 '萬般': 1.8,
 '萬分': 1.8,
 '萬萬': 1.8,
 '無比': 1.8,
 '無度': 1.8,
 '無可估量': 1.8,
 '無以復加': 1.8,
 '無以倫比': 1.8,
 '要命': 1.8,
 '要死': 1.8,
 '已極': 1.8,
 '已甚': 1.8,
 '異常': 1.8,
 '逾常': 1.8,
 '賊': 1.8,
 '之極': 1.8,
 '之至': 1.8,
 '至極': 1.8,
 '卓絕': 1.8,
 '最為': 1.8,
 '佼佼': 1.8,
 '郅': 1.8,
 '綦': 1.8,
 nan: 1.5,
 '最': 1.8,
 '不過': 1.6,
 '不少': 1.6,
 '不勝': 1.6,
 '慘': 1.6,
 '沉': 1.6,
 '沉沉': 1.6,
 '出奇': 1.6,
 '大為': 1.6,
 '多': 1.5,
 '多多': 1.6,
 '多加': 1.6,
 '多麼': 1.6,
 '分外': 1.6,
 '格外': 1.6

In [19]:
jieba.load_userdict('dict.txt.big')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.661 seconds.
Prefix dict has been built succesfully.


In [20]:
def hasOpposite(wordlist):
    opp = False

    for word in wordlist:
        if word in not_set:
            opp = True
    
    return opp

In [21]:
def getDegree(wordlist):
    degree = 1.0
    for word in wordlist:
        if word in degree_dict:
            degree = degree_dict[word]
    
    return degree

# 反義詞太多，很容易出現0分,改用加減分

In [22]:
def analyze (text):
    token = list(jieba.cut(text))
#     print(token)
    sum  = 0 
    for word in token:
        if word.lower() in positives_set:
            sum += 1
        elif word.lower() in negatives_set:
            sum -= 1
    
    sum = sum*getDegree(token)
    
    return sum

In [26]:
def sentiment_analysis(text):
    sentiment = 0
    score =  analyze(text)
    print('情感分數',score)
    if score >= 0:
        sentiment = 1
    elif score < 0.0:
        sentiment = 0
    return sentiment

In [23]:
score = {}
for i in data.index:
    comment = data.loc[i]['comment']
    score[i] = analyze(comment)
    print (score[i])

['這門', '課滿', '像', '演講', '，', '不用', '考試', '，', '老師', '常常', '跟', '我們', '分享', '他', '去', '阿拉伯', '工作', '的', '見聞', '，', '偶爾', '也', '會', '邀請', '一些', '在', '國外', '工作', '的', '人', '分享', '他們', '的', '趣事']
2.4
['不難', '，', '可以', '不去', '上課', '，', '最後', '一節', '繳交', '作業', '兼', '點名', '即可', '。']
1.0
['人', '真的', '很', '好']
1.6
['不用', '什麼', '基礎', '，', '人來', '就', '好', '，', '老師', '人', '很', '好', '，', '都', '是', '第二節', '上課', '才', '傳', '點名單', ' ', '，', '偶爾', '會', '直接', '點名']
8.0
['給分', '都', '很', '高']
1.6
['就', '聽', '老師', '講', '一些', '儒道', '思想', '，', '他', '也', '會問', '同學們', '哪裡', '有', '問題', '還有', '感想', '，', '所以', '要', '認真', '聽', '喔']
3.5999999999999996
['壓力', '不大']
0.0
['有點', '類似', '高中', '的', '公民']
0.0
['只要', '認真', '聽', '就', '會', '了', '，', '考前', '老師', '的', '複習', '一定', '要', '聽', '，', '回去', '看一看', '就', '90', '幾了']
4.0
['廢課', '一節', '，', '唯一', '有趣', '的', '是', '每節', '老師上課', '穿', '的', '衣服', '都', '很', '特別', '。']
3.2
['老師', '會點', '人', '回答', '看', '了', '電影', '有', '甚麼', '啟發', '?', '然後', '延伸', '到', '做人', '處世', '的', '道理', '和', '自

['一個月', '點名', '左右', '，', '沒有', '作業', '，', '考試', '只要', '對', '老師', '的', '申論', '提出', '質疑', '或', '辯論', '的', '感覺', '，', '寫越', '多', '分數', '越', '多', '，', '不', '需有', '法律', '底']
0.0
['出席', '都', '用', '簽到', '的', '，', '作業', '分數', '佔', '比較', '重', '，', '但是', '作業', '很', '簡單', '上網', '查一查', '就', '可以', '查', '到', '答案', '，', '作業', '內容', '越', '多分', '，', '分數', '越高']
6.4
['1', '.', ' ', '平時成績', ' ', '30%', ' ', ' ', '(', '小考', '和', '課堂', '參與', ')']
1.0
['成績', '分配', ' ', ':', ' ', '期中考', ' ', '30%', ' ', '、', '期末考', ' ', '30%', '、', '平時成績', ' ', '40%']
0.0
['分數', '基本上', '只要', '你', '上課', '有', '在', '聽', '，', '筆記', '都', '有', '抄', '，', '沒有', '九十', '也', '有', '八十', '，', '老師上課', '很', '幽默', '風趣', '，', '根本', '不會', '想', '翹', '課', '，', '重點', '是', '課程內容', '跟', '生活', '習習', '相關', '，', '因為', '有', '上過', '這門', '課', '，', '出車禍', '的', '時候', '才', '不會', '被', '別人', '佔便宜', '！', '！', '大', '推', '！']
11.200000000000001
['期中', '期末', '各', '佔', ' ', '35%', '、', '平時', '佔', ' ', '30%']
0.0
['期中', '期末', '各', '佔', '30%']
0.0
['有', '分組', '作業',

['期中', '簡單', '期末', '很難', '，', '有', '差異', '，', '建議', '期中', '拚高']
1.0
['有', '小考', '，', '幾乎', '每堂', '都', '有', '作業', '，', '會', '簽到', '或', '抽問', '當', '點名']
2.0
['老師', '人', '很', '好', '，', '課程', '進度', '慢', '，', '適合', '沒有', '基礎', '的', '人修', '，', '好過', '，', '偶爾', '點']
4.800000000000001
['好過', '，', '李', '90']
1.0
['好過', '、', '上課', '無聊', '、', '講課', '很', '小聲']
0.0
['教得', '好', '、', '滿嚴', '的', '、', '不好過', '、', '萬', '40', '、', 'but', '學', '很多']
0.0
['逢甲', '日', '文教', '的', '次', '好', '的', '老師', '，', '只是', '老師', '發音', '和', '張雪玉', '老師', '與', '日籍', '老師', '比', '起來', '有些', '差異', '，', '但是', '老師', '會', '用']
3.5999999999999996
['英文', '來講', '解', '日文', '文法', '，', '對', '初學者', '來說', '可能', '比較', '好', '懂', '。']
4.800000000000001
['這', '學期開始', '不', '需交', '報告', '，', '作業', '只要', '寫', '課本', '裡的', '就', '好', '，', '跟', '著', '步調', '走', '，', '基本上', '不難']
4.800000000000001
['教得', '很', '好', '、', '很', '嚴格', '、', '非常', '不好過', '、', '張', '30', '、', 'but', '學', '很多']
0.0
['逢甲', '日', '文教', '的', '最好', '，', '出身', '日本', '東北大學', '國語', '學

['上課', '很', '難懂', '，', '要', '自己', '讀', '，', '會', '有', '小考', '，', '通常', '一當', '都', '是', '半班', '以上', '，', '比鬼', '之', '大刀', '更', '可怕']
2.8
['教學', '不錯', '，', '會', '點名', '，', '有', '小考', '作業']
2.0
['大刀', '，', '刀', '起來', '完全', '不', '調分', '，', '上課', '要', '認真', '!']
5.4
['講課', '聲音', '會', '大會', '小', '，', '簽到', '，', '必修', '，', '非好過', '課程']
1.0
['很', '認真', '的', '老師', '，', '不要', '太', '混', '都', '會過']
0.0
['面惡心善', '，', '上課', '別太', '囂張']
-1.0
['教得', '很', '好', '，', '考試', '好', '很細', '，', '要', '很', '認真', '，', '不', '難過', '，', '高分', '很難', '。']
4.800000000000001
['工程', '數學', '，', '教', '的', '超好', '，', '還有', '錄', 'DVD', '可以', '看', '，', '一次', '聽不懂', '可以', '看', '很', '多次', '。']
0.0
['上課', '一定', '要', '抄', '筆記', '，', '睡覺', '、', '玩手', '機會', '被', '轟出去', '，', '老師', '秉持', '「', '你', '對', '老師', '好', '，', '老師']
6.4
['就', '會', '對', '你', '善良', '。', '你', '對', '老師', '不好', '，', '老師', '就', '會', '在', '期末考', '對', '你', '報仇', '。', '」', '老師', '對外', '系', '或']
5.0
['上', '暑修', '時會', '變大', '佛', '。']
2.0
['普通', '物理', '，', '教', '的', '超好'

In [24]:
data['score'] = data.index.map(score)

# 出來的結果，但是發現效果不太好，太多反義詞，或許學生覺的還可以

# 如：老師很無聊，無聊反義，但還可以接受

In [25]:
data

,comment,course,teacher,score
0,這門課滿像演講，不用考試，老師常常跟我們分享他去阿拉伯工作的見聞，偶爾也會邀請一些在國外工作...,阿拉伯世界-生活與經貿,詹授合,2.4
1,不難，可以不去上課，最後一節繳交作業兼點名即可。,通識-毒化物,忘了抱歉,1.0
2,人真的很好,實用營養學,何麗齡,1.6
3,不用什麼基礎，人來就好，老師人很好，都是第二節上課才傳點名單 ，偶爾會直接點名,國際關係,林賢治,8.0
4,給分都很高,行政學,黎淑慧,1.6
5,就聽老師講一些儒道思想，他也會問同學們哪裡有問題還有感想，所以要認真聽喔,儒道思想學情緒管理,王祖淼,3.6
6,壓力不大,智慧城市與行動應用,陳奕靜,0.0
7,有點類似高中的公民,經濟與生活,陳俊卿,0.0
8,只要認真聽就會了，考前老師的複習一定要聽，回去看一看就90幾了,營養與健康,何麗齡,4.0
9,廢課一節，唯一有趣的是每節老師上課穿的衣服都很特別。,世界音樂文化,盧韋聿,3.2


In [26]:
data.to_csv('comment_score.csv',index=0)

In [29]:
data = pd.read_csv('comment_score.csv',index_col='teacher')

In [35]:
teacher_index = data.index

In [36]:
list(teacher_index)

['詹授合',
 '忘了抱歉',
 '何麗齡',
 '林賢治',
 '黎淑慧',
 '王祖淼',
 '陳奕靜',
 '陳俊卿',
 '何麗齡',
 '盧韋聿',
 '楊裕仁',
 '楊郁仁',
 '賴奇厚',
 '葉美玲',
 '李允中',
 '紀文章',
 '劉南英',
 '吳珮瑄',
 '邵祖威',
 '潘才學',
 '楊裕仁',
 '葉怡巖',
 '黃圻文',
 '楊美美',
 '王祥齡',
 '金尚浩',
 '林賢治',
 '戴瑞坤',
 '彭妙卿',
 '黃圻文',
 '劉文甦',
 '朱文祺',
 '劉杜鎮',
 '經觀榮',
 '黎淑慧',
 '黃登煌',
 '吳秀英',
 '潘才學',
 '楊美美',
 '蔡曉伶',
 '黃智瑛',
 '梁煌儀',
 '黎淑慧',
 '滿肇怡',
 '李元璋',
 '陳秀文',
 '羅珮瑩',
 '楊裕仁',
 '李舒亭',
 '吳元豐',
 '楊裕仁',
 '只有這一位',
 '何麗齡',
 '彭瑞芝',
 '葉怡嚴',
 '陳上元',
 '黎淑慧',
 '李元璋',
 '張子揚',
 '楊裕仁',
 '張峰碩',
 '潘才學',
 '黃登煌',
 '黃志偉',
 '袁維勵',
 '林賢治',
 '馬彥彬',
 '蔡振志',
 '張瑞芬',
 '王祥齡',
 '蔡宜宏',
 '潘才學',
 '張漢威',
 '何麗齡',
 '宋明弘',
 '余老師，真名忘了抱歉',
 '王婉倫',
 '林彩玉',
 '張漢威',
 '陳至還',
 '高夢定',
 '楊美美',
 '許義忠',
 '林暘桂',
 '李元恕',
 '顏秀吉',
 '許慧雯',
 '吳東憲',
 '劉宗欣',
 '陳宏賓',
 '黃正一',
 '王雅慧',
 '林明洵',
 '林震燦',
 '曾柏興',
 '劉樹澤',
 '徐培晃',
 '林宗志',
 '陳青文',
 '張景星',
 '鄭佩玉',
 '黃喚堂',
 '林彩玉',
 '張雪玉',
 '林立謙',
 '林倖如',
 '吳如娟',
 '張漢威',
 '張育瑋',
 '林慧洙',
 '盧裕山',
 '伍昱睿',
 '陳立安',
 '廖重宏',
 '李文傳',
 '洪英雪',
 '劉士民',
 '林盈萱',
 '徐慧敏',
 '簡正儀',
 '施筱雲',
 '羅仙法',
 '陳奎佑',
 '劉南英'

In [38]:
teacher_index = list(set(teacher_index))

In [40]:
teacher_index

[nan,
 '李人先',
 '吳瑞雲',
 '簡榮聰',
 '馬偉成',
 '許家得',
 '曾鹿鳴',
 '羅珮瑩',
 '李寶玲',
 '張瑞麟',
 '洪裕勝',
 '黃焜炎',
 '洪國仁',
 '張勁燕',
 '陳裕益',
 '鍾懿萍',
 '蔡蕙芳',
 '杜方中',
 '黃喚堂',
 '林志煌',
 '寇敦智',
 '林國貴',
 '林暘桂',
 '洪維志',
 '朱文光',
 '陳坤森',
 '康照宗',
 '施筱雲',
 '辜文元',
 '林震燦',
 '劉志宏',
 '楊水源',
 '吳俊哲',
 '林韻柔',
 '林志敏',
 '劉明傑',
 '楊賀雯',
 '陳錫桓',
 '鄭淑芳',
 '馬以謹',
 '張森河',
 '李昌麟',
 '黎自奮',
 '林金福',
 '童翔新/張立德',
 '楊炳章',
 '周翊雯',
 '王益文',
 '楊文祿',
 '蘇雲華',
 '崔征國',
 '經觀榮',
 '楊璟琦',
 '許釗諭',
 '郭靜怡',
 '劉仲倫',
 '張清良',
 '楊郁仁',
 '陳益生',
 '徐瑞玲',
 '吳恆惠',
 '龐寶宏',
 '廖崇斐',
 '王振軒',
 '鄭春福',
 '張雪玉',
 '許禎娟',
 '施志欣',
 '魯漢平',
 '黃聰輝',
 '廖時三',
 '蔡佩珊',
 '許聖富',
 '莊建中',
 '魏猷恩',
 '朱心蘅',
 '江向才',
 '林季芸',
 '張寶蓉',
 '張沐雲',
 '鐘文伶',
 '林昱成',
 '徐慧敏',
 '藍文蒂',
 '邵祖威',
 '陳志瀅',
 '張景星',
 '曹秀惠',
 '陳志強',
 '郭錫陽',
 '陳星百',
 '張淑英',
 '袁維勵',
 '黎淑慧',
 '陳翠蘭',
 '麥科立',
 '葉光清',
 '蔡宜壽',
 '羅道正',
 '陳長城',
 '蔡宜宏',
 '王綺亞',
 '謝孟勳',
 '李綉玲',
 '汪浩',
 '羅榮源',
 '吳秀英',
 '劉平文',
 '林金杉',
 '何信瑩',
 'Murphy',
 '彭瑞芝',
 '陳依依',
 '張瑞元',
 '劉鼎清',
 '江仰婉',
 '林昆明',
 '林憲鋒',
 '鄭竹君',
 '李時銘',
 '賈凱傑',
 '楊明憲',
 '勞安妮',
 '強本高濤系列',
 

In [61]:
teacher_df = pd.DataFrame(columns = ["name", "score","recommend"])
for j,name in enumerate(teacher_index[1:]):
    score = 0
    for i in data.loc[name].score:
        try:
            score += float(i)
        except:
            print('error')
    if score<=0:
        teacher_df.loc[j] = [name,score,'不推']
    elif score>20:
        teacher_df.loc[j] = [name,score,'大推']
    else:
        teacher_df.loc[j] = [name,score,'推']

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [103]:
teacher_df

,name,score,recommend
0,李人先,1.0,推
1,吳瑞雲,15.0,推
2,簡榮聰,2.0,推
3,馬偉成,11.4,推
4,許家得,0.0,不推
5,曾鹿鳴,4.0,推
6,羅珮瑩,1.0,推
7,李寶玲,8.6,推
8,張瑞麟,6.0,推
9,洪裕勝,1.0,推


In [96]:
engine = create_engine('sqlite:///comment.db')

In [68]:
teacher_df.to_sql(name = 'teacher' ,con=engine, index=False, if_exists='append')

In [105]:
data = pd.read_csv('comment_score.csv')

In [107]:
data.columns = ['name','course','comment','score']

In [108]:
data.to_sql(name = 'comment' ,con=engine, index=False, if_exists='append')

In [77]:
engine = create_engine('sqlite:///database.db')

In [78]:
data = pd.read_sql('select * from course',con=engine)

In [89]:
data = data.drop(columns=['year','sub_credit'])

In [90]:
data

,scr_selcode,sub_id3,sub_name,scj_scr_mso,dept,unit
0,3241,GTEA001,教學學習實務(一),選修,通識中心,通識中心
1,3242,UNIV902,班級活動,必修,通識中心,全校國際生大一不分系學士班
2,3243,ATHL101,體育(一),必修,通識中心,全校國際生大一不分系學士班
3,3244,CHIN168,大學國文(一),必修,通識中心,全校國際生大一不分系學士班
4,3245,GBCP101,公民參與,必修,通識中心,全校國際生大一不分系學士班
5,3246,GBSP101,社會實踐,必修,通識中心,全校國際生大一不分系學士班
6,3247,PMP104,專業領域概述,必修,通識中心,全校國際生大一不分系學士班
7,3248,PMP101,溝通表達技巧(一),必修,通識中心,全校國際生大一不分系學士班
8,3249,PMP103,跨領域專題實作,必修,通識中心,全校國際生大一不分系學士班
9,3250,ECON101,經濟學(一),必修,通識中心,全校國際生大一不分系學士班


In [94]:
data.columns = ['choose_id','co_id','co_name','master_ornot','dept','unit']

In [101]:
data

,choose_id,co_id,co_name,master_ornot,dept,unit
0,3241,GTEA001,教學學習實務(一),選修,通識中心,通識中心
1,3242,UNIV902,班級活動,必修,通識中心,全校國際生大一不分系學士班
2,3243,ATHL101,體育(一),必修,通識中心,全校國際生大一不分系學士班
3,3244,CHIN168,大學國文(一),必修,通識中心,全校國際生大一不分系學士班
4,3245,GBCP101,公民參與,必修,通識中心,全校國際生大一不分系學士班
5,3246,GBSP101,社會實踐,必修,通識中心,全校國際生大一不分系學士班
6,3247,PMP104,專業領域概述,必修,通識中心,全校國際生大一不分系學士班
7,3248,PMP101,溝通表達技巧(一),必修,通識中心,全校國際生大一不分系學士班
8,3249,PMP103,跨領域專題實作,必修,通識中心,全校國際生大一不分系學士班
9,3250,ECON101,經濟學(一),必修,通識中心,全校國際生大一不分系學士班


In [102]:
data.to_sql(name = 'course' ,con=engine, index=False, if_exists='append')

In [109]:
engine = create_engine('sqlite:///database.db')
data = pd.read_sql('select * from teacher where teacher = "陳德生"',con=engine)

In [110]:
data

,teacher,course,department,star,a,b,c
0,陳德生,數位系統設計,資訊系必修,5/5 分,學期前老師會公佈評分方式，最後調分時會整體一起加，然後50~59的最後總成績為60分。,小考的話會依照教到的進度決定，基本上都是講課內容的變化題。,基本上都是課本內容的講解，要先修過邏輯設計才會比較懂整個課程。
1,陳德生,邏輯設計,資工系必修,5/5 分,老師最後把50-60分的學生全部變60，老師好,認真上課聽講，遇到困難尋求幫助，保證期末pass,不需基礎，但如果能先預習就很棒了
